In [1]:
import os
import sys
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.sequence import pad_sequences

sys.path.insert(0, r"../utilities")

from utils import *
from params import *
from Seq2SeqAttentionTrainer import Seq2SeqAttentionTrainer

tf.__version__

'2.0.0'

In [2]:
data_dir = "../data"
en_lines, fr_lines = read_data_files(data_dir, ("small_vocab_en", "small_vocab_fr"))

#data = read_data(os.path.join(data_dir, "fra-eng"), "fra.txt")

#en_lines, fr_lines = list(zip(*data))
#en_lines, fr_lines = shuffle(en_lines, fr_lines)

#en_lines = en_lines[:30000]
#fr_lines = fr_lines[:30000]

en_lines = [normalize(line) for line in en_lines]
fr_lines = [normalize(line) for line in fr_lines]

en_train, en_test, fr_train, fr_test = train_test_split(en_lines, fr_lines, shuffle=True, test_size=0.1)

en_lines = en_test
fr_lines = fr_test

# creating tokenizers
en_tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (en for en in en_train), target_vocab_size=2**13)

fr_tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (fr for fr in fr_train), target_vocab_size=2**13)

print("en_tokenizer size ", en_tokenizer.vocab_size)
print("fr_tokenizer size ", fr_tokenizer.vocab_size)

# train dataset
fr_train_in = [[fr_tokenizer.vocab_size] + fr_tokenizer.encode(line) for line in fr_train]
fr_train_out = [fr_tokenizer.encode(line) + [fr_tokenizer.vocab_size+1] for line in fr_train]

fr_train_in = pad_sequences(fr_train_in, padding='post')
fr_train_out = pad_sequences(fr_train_out, padding='post')

# test dataset
fr_test_in = [[fr_tokenizer.vocab_size] + fr_tokenizer.encode(line) for line in fr_test]
fr_test_out = [fr_tokenizer.encode(line) + [fr_tokenizer.vocab_size+1] for line in fr_test]

fr_test_in = pad_sequences(fr_test_in, padding='post')
fr_test_out = pad_sequences(fr_test_out, padding='post')

en_train = [en_tokenizer.encode(line) for line in en_train]
en_test = [en_tokenizer.encode(line) for line in en_test]

en_train = pad_sequences(en_train, padding='post')
en_test = pad_sequences(en_test, padding='post')

reading data from  ../data/small_vocab_en
reading data from  ../data/small_vocab_fr
en_tokenizer size  542
fr_tokenizer size  709


In [3]:
trainer = Seq2SeqAttentionTrainer(batch_size=BATCH_SIZE, 
                                  lstm_size=LSTM_SIZE, 
                                  embedding_size=EMBEDDING_SIZE, 
                                  tokenizers=[en_tokenizer, fr_tokenizer],
                                  predict_every=5)

In [4]:
losses, accuracy= trainer.train(train_data=[en_train, fr_train_in, fr_train_out], 
                                test_data=[en_test, fr_test_in, fr_test_out], 
                                prediction_data=[en_lines, fr_lines], 
                                epochs=20, 
                                attention_type="concat")

Number of devices: 4
creating dataset...
prediction input :  california is usually snowy during october but it is relaxing in august . 
prediction output:  californie est generalement enneigee en octobre mais il est relaxant au mois d aout . 
training from scratch
starting training with 20 epochs with prediction each 5 epoch
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Epoch 1 training Loss 0.8785 Accuracy 0.3888  test Loss 0.4425 Accuracy 0.4556
Saving checkpoint for epoch 0: ./checkpoints/Seq2SeqAttention/ckpt-1
----------------------------PREDICTION----------------------------
Predicted:  [<tf.Tensor: id=300839, shape=(1, 1, 13), dtype=float32, numpy=
array([[[2.7673268e-01, 2.4809645e-01, 1.7862366e-01, 1.2725733e-01,
         8.8961758e-02, 5.9171915e-02, 1.4649533e-02, 4.9717911e-03,
         1.0557234e-03, 2.8343583e-04, 1.0302673e-04, 5.7459558e-05,
         3.5213037e-05]]], dtype=float32)>, <tf.Tensor: id=301018, shape=(1, 1, 13), dtype=float32, numpy=
array([[[0.10927676, 0.11739983, 0.12743676, 0.12478688, 0.11346258,
         0.10429692, 0.07168653, 0.06264309, 0.04893987, 0.04122101,
         0.03262901, 0.02578465, 0.02043615]]], dtype=float32)>, <tf.Tensor: id=301197, shape=(1, 1, 13), dtype=float32, numpy=
array([[[2.8182485e-04, 2.9419368e-04, 3.3036666e-04, 3.4846086e-04,
         3.4604076e-04, 3.5897130e-04, 4.0688185e-04, 5.9199

AxisError: axis 1 is out of bounds for array of dimension 0

In [ ]:
fig = plt.figure()
fig_plot = fig.add_subplot()
fig_plot.plot(train_losses, label="train_loss")
fig_plot.plot(test_losses, label="test_loss")
fig_plot.legend(loc="upper right")
fig_plot.set_xlabel("epoch")
fig_plot.set_ylabel("loss")
fig_plot.grid(linestyle="--")
fig.savefig("losses_plot.png")

In [ ]:
fig = plt.figure()
fig_plot = fig.add_subplot()
fig_plot.plot(train_accuracyVec, label="train_accuracy")
fig_plot.plot(test_accuracyVec, label="test_accuracy")
fig_plot.legend(loc="lower right")
fig_plot.set_xlabel("epoch")
fig_plot.set_ylabel("accuracy")
fig_plot.grid(linestyle="--")
fig.savefig("accuracy_plot.png")

In [ ]:
trainer.translate("What are you doing?")